In [1]:
!pip install seaborn | wc -l
!pip install matplotlib | wc -l

14


The syntax of the command is incorrect.


In [2]:
CLASS_LIST = ["Tee","Tank","Dress","Shorts","Skirt","Jumpsuit","Sweater","Blazer","Striped","Cardigan","Blouse","Romper","Sweatpants","Jacket"]
for idx, val in enumerate(CLASS_LIST):
    print("%i/%s" % (idx, val))

0/Tee
1/Tank
2/Dress
3/Shorts
4/Skirt
5/Jumpsuit
6/Sweater
7/Blazer
8/Striped
9/Cardigan
10/Blouse
11/Romper
12/Sweatpants
13/Jacket


In [3]:
import tensorflow.keras as keras
import os
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))
os.chdir('C:\\Users\\mdc20')
from keras.models import load_model
model = load_model('best_model_14_class_lr@0.0001-1665971573.2646348.hdf5')

In [5]:
datasetdir = 'E:\\img_highres'
os.chdir(datasetdir)

from msilib.schema import Directory
from random import shuffle
import tensorflow as tf
import numpy as np
from keras.applications.vgg16 import VGG16, preprocess_input


from tensorflow.keras.preprocessing.image import ImageDataGenerator
BATCH_SIZE = 8
IMG_SHAPE = (512,512)


from collections import defaultdict
import pandas as pd
datasetdir = 'E:\\img_highres'
def constructImageClassDataFrame(shape, min_class_occurence = 1, class_list = None):
    # Args: 
    #   Shape: Image shape (2D)
    #   min_class_occurence: number of times class must occur in labeled dir in order to add it to the final sclass list
    sub_dirs = [d for d in os.listdir(os.getcwd()) if os.path.isdir(os.path.join(datasetdir, d))]
    
    if class_list is None:
        classes = defaultdict(lambda: 0)

        for sub_dir in sub_dirs:
            labels = sub_dir.split('_')
            for label in labels:
                classes[label] += 1
        top_k_classes = [cls for cls in classes if classes[cls] >= min_class_occurence]
        label_classes = top_k_classes    
    else:
        classes = class_list
        label_classes = classes

    
    arr = [[labeled_dir, file] for labeled_dir in sub_dirs for file in os.listdir(labeled_dir) ]
    print("Done parsing through directories")
    df = pd.DataFrame(data=arr, columns=["folder","filename"])



    df['filename'] = df['folder'] + '/' + df['filename']
    df['labels'] = df['folder'].apply(lambda x : [y for y in x.split('_') if y in label_classes] if len([y for y in x.split('_') if y in label_classes]) > 0 else None)

    print(np.shape(df))
    df = df[df.labels.notnull()]

    return df, label_classes

    

def DataLoad(shape, preprocessing): 
    '''Create the training and validation datasets for 
    a given image shape.
    '''
    imgdatagen = ImageDataGenerator(
        preprocessing_function = preprocessing,
        horizontal_flip = True, 
        validation_split = 0.1,
        rescale = 1.0/255, 
    )

    height, width = shape

    df, classes = constructImageClassDataFrame(shape, class_list=CLASS_LIST)

    # Modify dataframe to only take images with references to classes

    classes = list(classes)
    print("Num classes #")
    print(len(classes)) 
    # for subdir

    # train_dataset = imgdatagen.flow_from_dataframe(
    #     dataframe = df,
    #     directory = datasetdir,
    #     x_col="filename",
    #     y_col="labels",
    #     batch_size = batch_size,
    #     seed = 116,
    #     shuffle = True,
    #     class_mode="categorical",
    #     classes = classes,
    #     subset = 'training'
    # )
    val_dataset = imgdatagen.flow_from_dataframe(
        dataframe = df,
        directory = datasetdir,
        x_col="filename",
        y_col="labels",
        batch_size = BATCH_SIZE,
        seed = 42,
        target_size = IMG_SHAPE,
        shuffle = False,
        class_mode="categorical",
        classes = classes,
        subset = 'validation'
    )


    return 0, val_dataset, df

_, val_dataset, df = DataLoad((512,512), preprocessing=preprocess_input)


Done parsing through directories
(289212, 3)
Num classes #
14
Found 24638 validated image filenames belonging to 14 classes.


In [ ]:
import sklearn.metrics as skm
def get_indices_from_keras_generator(gen, batch_size):
    """
    Given a keras data generator, it returns the indices and the filepaths
    corresponding the current batch. 
    :param gen: keras generator.
    :param batch_size: size of the last batch generated.
    :return: tuple with indices and filenames
    """

    idx_left = (gen.batch_index - 1) * batch_size
    idx_right = idx_left + gen.batch_size if idx_left >= 0 else None
    indices = gen.index_array[idx_left:idx_right]
    filenames = [gen.filenames[i] for i in indices]
    return indices, filenames
from io import StringIO
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import classification_report

def get_confusion_matrix(y_test, y_hat, class_list, threshold = 0.5):
    # y_test is of form E = [e..e] where |E|=12 and e\in {0,1}
    # y_hat is of form P = [x..x] where 0 < x < 1 where P_x indicates probability of feature
    y_hat = np.where(y_hat > threshold,1,0)
    cm = confusion_matrix(y_test,y_hat,labels=class_list)
    return cm
# X_test, y_test = next(val_dataset)
# y_hat = model.predict(X_test)

class_list=CLASS_LIST

conf_matrix = np.array(np.zeros((14,2,2)))
print(np.shape(conf_matrix))
predicted_class_df = pd.DataFrame(columns=["filepath","class"])
MAX_ITERS = 500
it = 0
results_df = pd.DataFrame(columns=['class_identifier','precision','recall','f1-score','support','batch_number'],dtype=float)
for X_test,y_test in val_dataset:
    if it > MAX_ITERS:
        break
    if it % 10 == 0:
        print("%i/%i" % (it, MAX_ITERS))
    it = it + 1
    indices, filenames = get_indices_from_keras_generator(val_dataset,batch_size=4)
    y_hat = model.predict(X_test,verbose=0)
    y_hat = np.where(y_hat > 0.5,1,0)
    _
    # y_test_labels = np.argmax(y_test,axis=1)
    # y_hat_labels = np.argmax(y_hat, axis=1)

    # y_test_cm_input = [class_list[idx] for idx in y_test_labels]
    # y_hat_cm_input = [class_list[idx] for idx in y_hat_labels]
    cm = multilabel_confusion_matrix(y_test,y_hat,labels=range(0,14))
    # cm = get_confusion_matrix(y_test, y_hat, class_list=class_list)
    conf_matrix = np.add(conf_matrix, cm)
    # report = classification_report(y_test,y_hat,zero_division=0)
    # print(report)
    # rep_arr = report.split('\n')
    # rep_header = rep_arr[0].split()
    # rep_body = [row.split() for row in rep_arr[1:25]][1:]
# 
    # results_df.concat(rep_body)
    # df = pd.DataFrame(data=rep_body, columns=['class_identifier','precision','recall','f1-score','support'], index=class_list,dtype=float)
    # df['batch_number'] = it
    # results_df = pd.concat([results_df, df])

for class_name, conf_matrix in zip(class_list,conf_matrix):
    print(class_name)
    print(conf_matrix)
# results_df.sample(n=5)

In [ ]:

# results_df = results_df.drop(results_df[results_df.support == '0'].index)

# filtered_df = results_df[results_df.support != '0']
print(len(results_df))
# filtered_df.describe()

In [6]:
y_pred = model.predict(val_dataset)


3080/3080 [==============================] - 717s 227ms/step


In [7]:
print(np.shape(y_pred))

(24638, 14)


In [8]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer(classes=range(0,14), sparse_output=False)
y_true = val_dataset.classes
y_true_ohe = mlb.fit_transform(y_true)
print(np.shape(y_true_ohe))

(24638, 14)


In [9]:
def binarizeOutput(y_pred, threshold=0.5):
    y_pred_bin = np.where(y_pred > threshold,1,0)
    return y_pred_bin
y_pred_bin = binarizeOutput(y_pred, threshold=0.5)

In [ ]:
print(np.shape(y_pred_bin))
y_sum = np.sum(y_pred_bin,axis=1)
print(np.shape(y_sum))
y_sum_peek = y_sum[100:200]
print(y_sum_peek)
print(np.sum(y_sum))


 ### Classes
0/Tee
1/Tank
2/Dress
3/Shorts
4/Skirt
5/Hoodie
6/Jumpsuit
7/Sweater
8/Knit
9/Plaid
10/Blazer
11/Striped
12/Cardigan
13/Blouse
14/Dotted
15/Romper
16/Open-Back
17/Sweatpants
18/Leopard
19/Jacket

In [17]:
from sklearn.metrics import classification_report
SEARCH_THRESHOLD = False
t = 0.4
for threshold in [0.05, 0.10, 0.15, 0.20, 0.225, 0.25, 0.275, 0.3, 0.35, 0.4,0.45, 0.5]:
    if(not SEARCH_THRESHOLD): break
    y_pred_bin = binarizeOutput(y_pred, threshold=threshold)    

    c_report = classification_report(y_true=y_true_ohe, y_pred=y_pred_bin,zero_division=False)
    
    reports=c_report.split('\n')
    print("threshold:%f", threshold)
    print(reports[17])


y_pred_bin = binarizeOutput(y_pred, threshold=t)    
c_report = classification_report(y_true=y_true_ohe, y_pred=y_pred_bin,zero_division=False)
print(c_report)

              precision    recall  f1-score   support

           0       0.67      0.67      0.67      4369
           1       0.68      0.45      0.54      1976
           2       0.76      0.87      0.81      7581
           3       0.64      0.59      0.62      1687
           4       0.69      0.35      0.47      1425
           5       0.69      0.37      0.48       860
           6       0.54      0.21      0.30       996
           7       0.45      0.20      0.28       318
           8       0.37      0.21      0.26       685
           9       0.50      0.12      0.20       639
          10       0.46      0.49      0.47      2627
          11       0.61      0.26      0.37      1263
          12       0.74      0.31      0.43       222
          13       0.61      0.12      0.20       520

   micro avg       0.67      0.58      0.62     25168
   macro avg       0.60      0.37      0.44     25168
weighted avg       0.65      0.58      0.59     25168
 samples avg       0.56   

In [ ]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

# print(y_test)
y_test_labels = np.argmax(y_test,axis=1)
y_hat_labels = np.argmax(y_hat, axis=1)

# print(np.shape(y_test_labels))
# print(np.shape(y_hat_labels))
# cm = confusion_matrix(y_test_labels,y_hat_labels,labels=class_list)
df_cm = pd.DataFrame(conf_matrix, index = class_list, columns = class_list)
plt.figure(figsize = (10,7))
sns.heatmap(df_cm,annot=True)

In [ ]:
np.shape(val_dataset.filenames)

In [ ]:

indices, filenames = get_indices_from_keras_generator(val_dataset, 1024)
print(np.shape(indices))
print(np.shape(filenames))
print(indices[0])
print(filenames[0:4])